# 추천 시스템

간단한 넷플릭스 영화 추천 알고리즘 생각해보기
- 범주형 데이터를 다룬다.
액션물, 로맨스물, 스릴러물, 한드, 미드, 일드, 영드 등의 영화 item 데이터와 A, B, C 같은 user 데이터를 취급했습니다. 이러한 데이터는 연속적(continuous)이지 않고 이산적(discrete)입니다. 이를 범주형(categorical) 데이터라고 합니다.
- (숫자 벡터로 변환한 뒤) 유사도를 계산한다.
범주형 데이터들을 좌표에 나타내었는데, 좌표에 나타내기 위해서는 숫자로 이루어진 벡터(numerical vector)로 변환해야 합니다. 그리고 그 거리를 계산하여 유사도를 계산합니다.

## 범주형 데이터 -> 유사도 확인(코사인 유사도)

- 두 벡터값에 대해 코사인 값을 취해서 서로를 비교 -1 ~ 1 사이의 값을 가짐

![data](image/cos.png)

![data2](image/cos2.png)

### 간단한 코사인 유사도 구하기 실습

In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

t1 = np.array([1, 1, 1])
t2 = np.array([2, 0, 1])

def cos_sim(A, B):
	return dot(A, B)/(norm(A)*norm(B))
cos_sim(t1, t2)

0.7745966692414834

### using sklearn

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
# 입력값으로 2차원 배열을 받으므로 2차원 배열로 정의
t1 = np.array([[1, 1, 1]])
t2 = np.array([[2, 0, 1]])
cosine_similarity(t1,t2)

array([[0.77459667]])

** 다른 유사도 계산법 **
- 유클리드 거리
- 자카드 유사도
- 피어슨 상관계수 등

## 추천시스템 종류

1. 콘텐츠 기반 필터링(Content Based Filtering)
2. 협업 필터링(Collaborative Filtering)
    - 사용자 기반
    - 아이템 기반
    - 잠재요인 협업 필터링 (latent factor collaborative filtering) → 행렬 인수분해(matrix factorization)
3. Deep Learning 적용 or Hybrid 방식
![image](image/rec.png)

## Content Based Filtering